# Random Forest

In [ ]:
%load_ext autotime

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import numpy as np
import copy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

In [ ]:
import pandas as pd
df_annotations=  pd.read_csv('C:/Users/u0146965/OneDrive - KU Leuven/Sentiment analysis/full_cleaned_dataset.csv')
df_annotations.head() # to display the first 5 lines of loaded data

In [ ]:
def y_set(dataset_column):
    y= dataset_column
    y = y.replace('NEUTRAL','1')
    y = y.replace('POSITIVE','2')
    y = y.replace('NEGATIVE','0')
    for i in y:
        i=int(i)
    return y

In [ ]:
def cf_mat(y_test, y_pred):
    mat=confusion_matrix(y_test, y_pred)/len(y_test)
    return sns.heatmap(mat, annot=True)

In [ ]:
#max_feat --> sqrt, log2, None

In [ ]:
def rf(X, y_inputkolom, max_feat, n_gram, n_est=100, max_feat_rf='sqrt' ):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    #do TFIDF
    vect=TfidfVectorizer(max_features=max_feat, stop_words= stopwords.words('dutch'), token_pattern=r'\b[^\d\W][^\d\W]+\b', ngram_range=n_gram).fit(X_train)
    X_train= vect.transform(X_train)
    X_test= vect.transform(X_test)
    X_train=pd.DataFrame(X_train.toarray(), columns= vect.get_feature_names())
    X_test=pd.DataFrame(X_test.toarray(), columns= vect.get_feature_names())
    
    # Build a Random Forest
    model = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)

    # Predict the labels 
    y_predict = model.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', model.score(X_test, y_test))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    
    
    #return y_predict, y_test, log_reg

In [ ]:
def rf_val(X,y_inputkolom, max_feat, n_gram, n_est=100, max_feat_rf='sqrt'):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    vect=TfidfVectorizer(max_features=max_feat, stop_words= stopwords.words('dutch'), token_pattern=r'\b[^\d\W][^\d\W]+\b', ngram_range=n_gram).fit(X_train)
    X_train= vect.transform(X_train)
    X_val= vect.transform(X_val)
    X_train=pd.DataFrame(X_train.toarray(), columns= vect.get_feature_names())
    X_val=pd.DataFrame(X_val.toarray(), columns= vect.get_feature_names())
   
    # Build a Random Forest
    model = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)

    # Predict the labels 
    y_predict = model.predict(X_val)
    acc_score= model.score(X_val, y_val)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

# TFIDF

## Plain text

In [ ]:
plotje=[]
max_feat_list=[1000, 2000, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    acc_score=rf_val(df_annotations['text'], df_annotations['label'], i, (1,1))
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    acc_score=rf_val(df_annotations['text'], df_annotations['label'],8000 , i)

optimizing the number of trees and the number of considered features by each tree

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['text'], df_annotations['label'],8000 , (1,1) , i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['text'], df_annotations['label'],8000 , (1,1) , 200,  i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('number of estimaters')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

TFIDF and then rf with optimal parameters

In [ ]:
rf(df_annotations['text'], df_annotations['label'], 8000, (1,1), 200, 'log2' )

## Stemming

In [ ]:
plotje=[]
max_feat_list=[1000, 2000, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    acc_score=rf_val(df_annotations['processed_annotations_joined'].astype('U'), df_annotations['label'], i, (1,1))
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    rf_val(df_annotations['processed_annotations_joined'].astype('U'), df_annotations['label'],6000 , i)

optimizing the regularization term

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['processed_annotations_joined'].astype('U'), df_annotations['label'],6000 , (1,1) , i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['processed_annotations_joined'].astype('U'), df_annotations['label'],6000 , (1,1) , 150,  i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('number of estimaters')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

TFIDF with optimal parameters with extra preprocessing

In [ ]:
rf(df_annotations['processed_annotations_joined'].astype('U'), df_annotations['label'], 6000, (1,1), 150 ,'log2')

## Lemmatization

In [ ]:
plotje=[]
max_feat_list=[1000, 2000, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    acc_score=rf_val(df_annotations['processed_annotations_lemmatized_joined'].astype('U'), df_annotations['label'],i , (1,1))
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    rf_val(df_annotations['processed_annotations_lemmatized_joined'].astype('U'), df_annotations['label'],12000, (1,1))

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['processed_annotations_lemmatized_joined'].astype('U'), df_annotations['label'], 12000, (1,2),i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['processed_annotations_lemmatized_joined'].astype('U'), df_annotations['label'],12000 , (1,2) , 150,  i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
rf(df_annotations['processed_annotations_lemmatized_joined'].astype('U'), df_annotations['label'], 12000, (1,2), 150 , 'log2' )

## Word2Vec selftrained

In [ ]:
def sent(data_column):
    sentences=[]
    for i in data_column:
        if type(i)!=float:
            words_in_sentence=i.split()
            sentences.append(words_in_sentence)
    return sentences

In [ ]:
def rf_val(X,y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, n_est=100, max_feat_rf='sqrt'):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        #a = np.array(l)
        #print(a)
        #res = np.average(a, axis=0)
        #lijst=list(res)
        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    Word2Vec_text_av=[]
    for i in X_val:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        #a = np.array(l)
        #print(a)
        #res = np.average(a, axis=0)
        #lijst=list(res)
        Word2Vec_text_av.append(avg)
    X_val=pd.DataFrame(Word2Vec_text_av)
   
    # Build a logistic regression
    rf = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = rf.predict(X_val)
    acc_score= rf.score(X_val, y_val)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

In [ ]:
def rf(X,y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, n_est=100, max_feat_rf='sqrt'):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    Word2Vec_text_av=[]
    for i in X_test:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X_test=pd.DataFrame(Word2Vec_text_av)
    
    # Build a logistic regression
    rf = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)

    # Predict the labels 
    y_predict = rf.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', rf.score(X_test, y_test))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    
    
    #return y_predict, y_test, log_reg

## Textified

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    acc_score=rf_val(df_annotations['text'],df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

so we choose vector_size= 100

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    acc_score=rf_val(df_annotations['text'],df_annotations['label'], 300,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje) #wind=5, min_c=1, work=4, sg=1, alpha= 0.025,

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

we kiezen window 15

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20]
for i in min_c:
    acc_score=rf_val(df_annotations['text'],df_annotations['label'], 300,15,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

dus we kiezen min_count 5

In [ ]:
plotje=[]
for i in [0,1]:
    acc_score=rf_val(df_annotations['text'],df_annotations['label'], 300,15,5,4,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

dus sgd=1

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    acc_score=rf_val(df_annotations['text'],df_annotations['label'], 300,15,5,16,1,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

Dus we kiezen 0.15

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['text'],df_annotations['label'], 300,15,5,16,1,0.05,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val(df_annotations['text'],df_annotations['label'], 300,15,5,16,1,0.05,150 ,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
rf(df_annotations['text'],df_annotations['label'], 300,15,5,16,1,0.05,150 ,'sqrt')

## Lemmatized

In [ ]:
def rf_val_lem(X,y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, n_est=100, max_feat_rf='sqrt'):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]

        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    
    Word2Vec_text_av=[]
    for i in X_val:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
    
        Word2Vec_text_av.append(avg)
    X_val=pd.DataFrame(Word2Vec_text_av)
   
    # Build a logistic regression
    rf = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)

    # Predict the labels 
    y_predict = rf.predict(X_val)
    acc_score= rf.score(X_val, y_val)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

In [ ]:
def rf_lem(X,y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, n_est=100, max_feat_rf='sqrt'):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]

        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    
    Word2Vec_text_av=[]
    for i in X_test:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
    
        Word2Vec_text_av.append(avg)
    X_test=pd.DataFrame(Word2Vec_text_av)
   
    # Build a logistic regression
    rf = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)

    # Predict the labels 
    y_predict = rf.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', rf.score(X_test, y_test))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    acc_score=rf_val_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    acc_score=rf_val_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], 100,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20]
for i in min_c:
    acc_score=rf_val_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], 100,15,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    acc_score=rf_val_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], 100,15 ,7 ,16,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    acc_score=rf_val_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], 100,15 , 7,16,1,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], 100,15,7,16,1,0.1,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    acc_score=rf_val_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], 100,15,7,16,1,0.1,200 ,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
rf_lem(df_annotations['processed_annotations_lemmatized_joined'],df_annotations['label'], 100,15,7,16,1,0.1,200 ,'sqrt')

# Word2Vec using pretrained embeddings

In [ ]:
import gensim.downloader as api
corpus = api.load('text8')
model = Word2Vec(corpus)

In [ ]:
import gensim.downloader as api
corpus = api.load('text8')
model = Word2Vec(corpus)

In [ ]:
import gensim

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/u0146965/OneDrive - KU Leuven/Sentiment analysis/model.bin', binary=True)

In [ ]:
def W2V_pretrained_emb(data):
    Word2Vec_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                if word in model.key_to_index:
                    l.append(list(model.get_vector(word)))
                else:
                    l.append([0]*300)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

In [ ]:
# Import the accuracy and confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

def rf(X, y_inputkolom, n_est=100, max_feat_rf='sqrt'):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Build a logistic regression
    rf = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = rf.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Micro F1 measure is', metrics.f1_score(new, y_pred, average='micro'))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Micro precsion is',metrics.precision_score(new, y_pred, average='micro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Micro recall is',metrics.recall_score(new, y_pred, average='micro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', rf.score(X_test, y_test))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    
    
    #return y_predict, y_test, log_reg

In [ ]:
def rf_val(X,y_inputkolom, n_est=100, max_feat_rf='sqrt'):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    

    # Build a logistic regression
    rf = RandomForestClassifier(n_estimators=n_est, max_features= max_feat_rf).fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = rf.predict(X_val)
    acc_score= rf.score(X_val, y_val)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

## Textified

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    X=W2V_pretrained_emb(df_annotations['text'])
    acc_score=rf_val(X,df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    X=W2V_pretrained_emb(df_annotations['text'])
    acc_score=rf_val(X,df_annotations['label'], 150 ,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
X=W2V_pretrained_emb(df_annotations['text'])
acc_score=rf(X, df_annotations['label'], 150,'sqrt')

## Lemmatized

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
    acc_score=rf_val(X,df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
    acc_score=rf_val(X,df_annotations['label'], 200 ,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
acc_score=rf(X, df_annotations['label'], 200, 'sqrt')

# Fasttext using pretrained embeddings

In [ ]:
#https://fasttext.cc/docs/en/support.html#building-fasttext-python-module
#https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
import fasttext
import fasttext.util

In [ ]:
fasttext.util.download_model('nl', if_exists='ignore')  # English

In [ ]:
model = fasttext.load_model('C:/Users/u0146965/OneDrive - KU Leuven/Sentiment analysis/cc.nl.300.bin')

In [ ]:
def fasttext_pretrained_emb(data):
    fasttext_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                l.append(list(model.get_word_vector(word)))
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        fasttext_text_av.append(avg)
    X=pd.DataFrame(fasttext_text_av)
    return X

## Textified

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    X=fasttext_pretrained_emb(df_annotations['text'])
    acc_score=rf_val(X,df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    X=fasttext_pretrained_emb(df_annotations['text'])
    acc_score=rf_val(X,df_annotations['label'], 150 ,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
X=fasttext_pretrained_emb(df_annotations['text'])
acc_score=rf(X, df_annotations['label'], 150, 'sqrt')

## Lemmatized

In [ ]:
plotje=[]
n_est=[10, 50, 100, 150, 200]
for i in n_est:
    print('the regularisation term is', i)
    X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
    acc_score=rf_val(X,df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('regularisation term')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est=['sqrt', 'log2']
for i in n_est:
    print('the regularisation term is', i)
    X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
    acc_score=rf_val(X,df_annotations['label'], 200 ,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
acc_score=rf(X, df_annotations['label'], 200,'sqrt')